## Kabak Fiyatlandırması için Doğrusal ve Polinom Regresyon - Ders 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalli tarafından hazırlanan bilgi grafiği</figcaption>


<!--![Dasani Madipalli tarafından hazırlanan bilgi grafiği](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Giriş

Şimdiye kadar, bu derste kullanacağımız kabak fiyatlandırma veri setinden toplanan örnek verilerle regresyonun ne olduğunu keşfettiniz. Ayrıca bunu `ggplot2` kullanarak görselleştirdiniz. 💪

Artık ML için regresyona daha derinlemesine dalmaya hazırsınız. Bu derste, iki tür regresyon hakkında daha fazla bilgi edineceksiniz: *temel doğrusal regresyon* ve *polinom regresyon*, bu tekniklerin altında yatan matematikle birlikte.

> Bu müfredat boyunca, matematik bilgisi gereksinimini minimumda tutmayı hedefliyoruz ve diğer alanlardan gelen öğrenciler için erişilebilir hale getirmeye çalışıyoruz. Bu nedenle notlar, 🧮 matematiksel açıklamalar, diyagramlar ve kavramayı kolaylaştıracak diğer öğrenme araçlarına dikkat edin.

#### Hazırlık

Hatırlatma olarak, bu veriyi yükleyerek ona sorular sormayı amaçlıyorsunuz.

-   Kabak almak için en iyi zaman ne zaman?

-   Mini kabakların bir kasasının fiyatı ne kadar olabilir?

-   Kabakları yarım sepetlik sepetlerde mi yoksa 1 1/9 sepetlik kutularda mı almalıyım? Bu veriyi daha fazla incelemeye devam edelim.

Önceki derste, bir `tibble` (veri çerçevesinin modern bir yeniden tasarımı) oluşturdunuz ve orijinal veri setinin bir kısmını fiyatlandırmayı sepet başına standartlaştırarak doldurdunuz. Ancak bunu yaparak, yalnızca yaklaşık 400 veri noktası ve yalnızca sonbahar ayları için veri toplayabildiniz. Belki veriyi daha fazla temizleyerek onun doğası hakkında biraz daha ayrıntı elde edebiliriz? Göreceğiz... 🕵️‍♀️

Bu görev için aşağıdaki paketlere ihtiyacımız olacak:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/), veri bilimini daha hızlı, kolay ve eğlenceli hale getirmek için tasarlanmış bir [R paketleri koleksiyonudur](https://www.tidyverse.org/packages).

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) çerçevesi, modelleme ve makine öğrenimi için bir [paketler koleksiyonudur](https://www.tidymodels.org/packages).

-   `janitor`: [janitor paketi](https://github.com/sfirke/janitor), kirli verileri incelemek ve temizlemek için basit araçlar sağlar.

-   `corrplot`: [corrplot paketi](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html), değişkenler arasındaki gizli desenleri tespit etmeye yardımcı olmak için otomatik değişken sıralamasını destekleyen korelasyon matrisinde görsel bir keşif aracı sağlar.

Bu paketleri şu şekilde yükleyebilirsiniz:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Aşağıdaki script, bu modülü tamamlamak için gereken paketlere sahip olup olmadığınızı kontrol eder ve eksik olanları sizin için yükler.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Bu harika paketleri daha sonra yükleyip mevcut R oturumumuzda kullanılabilir hale getireceğiz. (Bu sadece bir örnekleme için, `pacman::p_load()` bunu zaten sizin için yapıyor)

## 1. Bir doğrusal regresyon çizgisi

1. Derste öğrendiğiniz gibi, doğrusal regresyon çalışmasının amacı, aşağıdakilere en uygun *çizgiyi* çizebilmektir:

-   **Değişken ilişkilerini göstermek**. Değişkenler arasındaki ilişkiyi göstermek.

-   **Tahminlerde bulunmak**. Yeni bir veri noktasının bu çizgiye göre nerede yer alacağını doğru bir şekilde tahmin etmek.

Bu tür bir çizgiyi çizmek için **En Küçük Kareler Regresyonu** adı verilen bir istatistiksel teknik kullanırız. `En küçük kareler` terimi, regresyon çizgisinin çevresindeki tüm veri noktalarının karelerinin alınması ve ardından toplanması anlamına gelir. İdeal olarak, bu toplam mümkün olduğunca küçük olmalıdır, çünkü daha az hata, yani `en küçük kareler` istiyoruz. Bu nedenle, en uygun çizgi, kare hataların toplamı için en düşük değeri veren çizgidir - bu yüzden adı *en küçük kareler regresyonu*.

Bunu yapmamızın nedeni, tüm veri noktalarımızdan en az toplam mesafeye sahip bir çizgi modellemek istememizdir. Ayrıca, büyüklüğüyle ilgilendiğimiz için terimleri toplama işleminden önce karesini alırız, yönüyle değil.

> **🧮 Matematiği Göster**
>
> Bu çizgi, *en uygun çizgi* olarak adlandırılır ve [bir denklemle](https://en.wikipedia.org/wiki/Simple_linear_regression) ifade edilebilir:
>
>     Y = a + bX
>
> `X`, '`açıklayıcı değişken` veya `tahmin edici`'dir. `Y`, '`bağımlı değişken` veya `sonuç`'tur. Çizginin eğimi `b` ve `a` ise y-keseni, yani `X = 0` olduğunda `Y`'nin değerini ifade eder.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "eğim = $y/x$")
    Jen Looper tarafından hazırlanan infografik
>
> İlk olarak, eğim `b` hesaplanır.
>
> Başka bir deyişle, ve kabak verilerimizin orijinal sorusuna atıfta bulunarak: "Bir ay boyunca bir kabak sepetinin fiyatını tahmin edin", `X` fiyatı ifade ederken, `Y` satış ayını ifade eder.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Jen Looper tarafından hazırlanan infografik
> 
> `Y` değerini hesaplayın. Eğer yaklaşık 4 dolar ödüyorsanız, bu Nisan olmalı!
>
> Çizgiyi hesaplayan matematik, çizginin eğimini göstermelidir, bu da aynı zamanda kesişim noktasına, yani `X = 0` olduğunda `Y`'nin konumuna bağlıdır.
>
> Bu değerlerin hesaplama yöntemini [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) web sitesinde gözlemleyebilirsiniz. Ayrıca, sayıların değerlerinin çizgiyi nasıl etkilediğini görmek için [bu En Küçük Kareler hesaplayıcısını](https://www.mathsisfun.com/data/least-squares-calculator.html) ziyaret edin.

Korkutucu değil, değil mi? 🤓

#### Korelasyon

Anlamanız gereken bir diğer terim, verilen X ve Y değişkenleri arasındaki **Korelasyon Katsayısı**dır. Bir dağılım grafiği kullanarak bu katsayıyı hızlıca görselleştirebilirsiniz. Verilerin düzgün bir çizgi üzerinde sıralandığı bir grafik yüksek korelasyona sahiptir, ancak X ve Y arasında her yere dağılmış veri noktalarına sahip bir grafik düşük korelasyona sahiptir.

İyi bir doğrusal regresyon modeli, En Küçük Kareler Regresyonu yöntemiyle ve bir regresyon çizgisiyle yüksek (1'e daha yakın, 0'dan uzak) bir Korelasyon Katsayısına sahip olan modeldir.


## **2. Verilerle dans: modelleme için kullanılacak bir veri çerçevesi oluşturma**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>@allison_horst tarafından yapılmış bir sanat eseri</figcaption>


<!--![@allison_horst tarafından yapılmış bir sanat eseri](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Gerekli kütüphaneleri ve veri setini yükleyin. Verileri, aşağıdaki alt küme verilerini içeren bir veri çerçevesine dönüştürün:

-   Sadece kile başına fiyatlandırılan kabakları alın

-   Tarihi bir aya dönüştürün

-   Fiyatı, yüksek ve düşük fiyatların ortalaması olarak hesaplayın

-   Fiyatı, kile miktarına göre fiyatlandırmayı yansıtacak şekilde dönüştürün

> Bu adımları [önceki derste](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb) ele almıştık.


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Saf bir macera ruhuyla, kirli verileri incelemek ve temizlemek için basit işlevler sağlayan [`janitor paketi`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor)'ni keşfedelim. Örneğin, verilerimiz için sütun adlarına bir göz atalım:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Daha iyisini yapabiliriz. Bu sütun adlarını [snake_case](https://en.wikipedia.org/wiki/Snake_case) kuralına dönüştürerek `janitor::clean_names` kullanarak `friendR` yapalım. Bu fonksiyon hakkında daha fazla bilgi edinmek için: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Çok düzenli 🧹! Şimdi, önceki derste olduğu gibi `dplyr` kullanarak verilerle bir dans! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Tebrikler! 👌 Artık yeni regresyon modelinizi oluşturabileceğiniz temiz ve düzenli bir veri setine sahipsiniz!

Bir dağılım grafiği ister misiniz?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Bir dağılım grafiği, elimizde yalnızca Ağustos'tan Aralık'a kadar olan ay verilerinin bulunduğunu hatırlatıyor. Doğrusal bir şekilde sonuçlara varabilmek için muhtemelen daha fazla veriye ihtiyacımız var.

Modelleme verilerimize tekrar bir göz atalım:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Bir kabağın `fiyatını`, karakter türündeki `şehir` veya `paket` sütunlarına dayanarak tahmin etmek isteseydik ne olurdu? Ya da daha basit bir şekilde, örneğin `paket` ve `fiyat` arasında (her iki girdinin de sayısal olması gerektiği) korelasyonu nasıl bulabilirdik? 🤷🤷

Makine öğrenimi modelleri, metin değerlerinden ziyade sayısal özelliklerle daha iyi çalışır, bu nedenle genellikle kategorik özellikleri sayısal temsillere dönüştürmeniz gerekir.

Bu, tahmin edicilerimizi bir modelin etkili bir şekilde kullanmasını kolaylaştıracak şekilde yeniden biçimlendirme yolunu bulmamız gerektiği anlamına gelir; bu sürece `özellik mühendisliği` denir.


## 3. Modeller için verileri tariflerle ön işleme 👩‍🍳👨‍🍳

Tahmin edici değerleri yeniden biçimlendirerek bir modelin bunları daha etkili kullanmasını sağlama faaliyetlerine `özellik mühendisliği` denir.

Farklı modellerin farklı ön işleme gereksinimleri vardır. Örneğin, en küçük kareler yöntemi `ay, çeşit ve şehir_adı gibi kategorik değişkenlerin kodlanmasını` gerektirir. Bu, basitçe `kategorik değerler` içeren bir sütunun, orijinal sütunun yerine geçen bir veya daha fazla `sayısal sütuna` dönüştürülmesini içerir.

Örneğin, verilerinizde aşağıdaki kategorik özellik bulunduğunu varsayalım:

|  şehir   |
|:--------:|
| Denver   |
| Nairobi  |
|  Tokyo   |

*Ordinal kodlama* uygulayarak her kategoriye benzersiz bir tam sayı değeri atayabilirsiniz, şöyle:

| şehir |
|:-----:|
|   0   |
|   1   |
|   2   |

Ve işte bunu verilerimize uygulayacağız!

Bu bölümde, verilerinizi modelinizi eğitmeden **önce** ön işleme konusunda size yardımcı olmak için tasarlanmış bir başka harika Tidymodels paketi olan [recipes](https://tidymodels.github.io/recipes/) paketini keşfedeceğiz. Temelde bir tarif, bir veri setine modelleme için hazır hale getirmek amacıyla hangi adımların uygulanması gerektiğini tanımlayan bir nesnedir.

Şimdi, tahmin edici sütunlardaki tüm gözlemler için benzersiz bir tam sayı atayarak verilerimizi modelleme için hazırlayan bir tarif oluşturalım:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Harika! 👏 İlk tarifimizi oluşturduk ve bu tarif bir sonucu (fiyat) ve buna karşılık gelen tahmin edicileri belirtiyor, ayrıca tüm tahmin edici sütunların bir dizi tam sayıya kodlanması gerektiğini ifade ediyor 🙌! Hadi bunu hızlıca parçalarına ayıralım:

-   `recipe()` çağrısı, bir formül ile birlikte, `new_pumpkins` verilerini referans alarak değişkenlerin *rollerini* tarife bildirir. Örneğin, `price` sütunu bir `outcome` rolüne atanmışken, diğer sütunlar bir `predictor` rolüne atanmıştır.

-   `step_integer(all_predictors(), zero_based = TRUE)` tüm tahmin edicilerin 0'dan başlayan bir numaralandırma ile bir dizi tam sayıya dönüştürülmesi gerektiğini belirtir.

Eminiz ki şu tür düşünceleriniz olabilir: "Bu çok havalı!! Ama ya tariflerin tam olarak beklediğim gibi çalıştığını doğrulamam gerekirse? 🤔"

Bu harika bir düşünce! Görüyorsunuz, tarifiniz bir kez tanımlandıktan sonra, veriyi gerçekten ön işlemek için gereken parametreleri tahmin edebilir ve ardından işlenmiş veriyi çıkarabilirsiniz. Tidymodels kullanırken genellikle bunu yapmanız gerekmez (birazdan normal yöntemi göreceğiz-\> `workflows`), ancak tariflerin beklediğiniz gibi çalıştığını doğrulamak için bir tür kontrol yapmak istediğinizde işe yarayabilir.

Bunun için iki ek fiile ihtiyacınız olacak: `prep()` ve `bake()`. Her zamanki gibi, [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) tarafından hazırlanan küçük R arkadaşlarımız bunu daha iyi anlamanıza yardımcı oluyor!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>@allison_horst tarafından yapılmış sanat eseri</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): bir eğitim setinden gerekli parametreleri tahmin eder ve bu parametreler daha sonra diğer veri setlerine uygulanabilir. Örneğin, belirli bir tahmin edici sütunu için hangi gözlem 0, 1, 2 gibi bir tam sayı ile atanacak.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): hazırlanmış bir tarifi alır ve işlemleri herhangi bir veri setine uygular.

Öyleyse, tariflerimizi hazırlayıp uygulayalım ve gerçekten doğrulayalım ki perde arkasında tahmin edici sütunlar önce kodlanacak, ardından bir model oluşturulacak.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 İşlenmiş veri `baked_pumpkins` tüm tahmin edicilerinin kodlandığını doğruladı, bu da tarif olarak tanımlanan ön işleme adımlarının beklendiği gibi çalışacağını gösteriyor. Bu, sizin için okumayı zorlaştırabilir ama Tidymodels için çok daha anlaşılır hale getirir! Hangi gözlemin ilgili bir tam sayıya eşlendiğini bulmak için biraz zaman ayırın.

Ayrıca, `baked_pumpkins` üzerinde hesaplamalar yapabileceğimiz bir veri çerçevesi olduğunu belirtmekte fayda var.

Örneğin, verilerinizdeki iki nokta arasında iyi bir korelasyon bulmaya çalışabiliriz, böylece potansiyel olarak iyi bir tahmin modeli oluşturabiliriz. Bunu yapmak için `cor()` fonksiyonunu kullanacağız. Fonksiyon hakkında daha fazla bilgi edinmek için `?cor()` yazabilirsiniz.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Görünüşe göre, Şehir ve Fiyat arasında yalnızca zayıf bir ilişki var. Ancak Paket ve Fiyatı arasında biraz daha iyi bir ilişki bulunuyor. Bu mantıklı, değil mi? Genelde, ürün kutusu ne kadar büyükse, fiyat da o kadar yüksek olur.

Bu sırada, tüm sütunların bir korelasyon matrisini `corrplot` paketi kullanarak görselleştirmeyi de deneyelim.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Çok daha iyi.

Bu verilerle şimdi sorulabilecek iyi bir soru şu olabilir: '`Belirli bir kabak paketi için hangi fiyatı bekleyebilirim?`' Haydi başlayalım!

> Not: **`pumpkins_prep`** tarifini **`new_data = NULL`** ile **`bake()`** ettiğinizde, işlenmiş (yani kodlanmış) eğitim verilerini elde edersiniz. Örneğin, başka bir veri setiniz (örneğin bir test seti) varsa ve bir tarifin onu nasıl ön işleyeceğini görmek istiyorsanız, **`pumpkins_prep`** tarifini **`new_data = test_set`** ile **`bake()`** etmeniz yeterlidir.

## 4. Doğrusal regresyon modeli oluşturun

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalli tarafından hazırlanan bilgi grafiği</figcaption>


<!--![Dasani Madipalli tarafından hazırlanan bilgi grafiği](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Artık bir tarif oluşturduğumuza ve verilerin uygun şekilde ön işleneceğini doğruladığımıza göre, şimdi şu soruyu yanıtlamak için bir regresyon modeli oluşturalım: `Belirli bir kabak paketi için hangi fiyatı bekleyebilirim?`

#### Eğitim seti kullanarak bir doğrusal regresyon modeli eğitin

Muhtemelen zaten fark etmişsinizdir, *price* sütunu `sonuç` değişkeni iken *package* sütunu `tahmin edici` değişkendir.

Bunu yapmak için, önce verileri %80'i eğitim setine ve %20'si test setine gidecek şekilde böleceğiz, ardından tahmin edici sütunu bir dizi tam sayıya kodlayacak bir tarif tanımlayacağız ve ardından bir model spesifikasyonu oluşturacağız. Tarifimizi hazırlayıp pişirmeyeceğiz çünkü verileri beklendiği gibi ön işleyeceğini zaten biliyoruz.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Harika iş çıkardınız! Artık bir tarifimiz ve bir model spesifikasyonumuz olduğuna göre, bunları bir araya getirip, önce veriyi ön işleme (arka planda hazırlık + pişirme), ardından ön işlenmiş veri üzerinde modeli eğitme ve potansiyel olarak son işlem aktivitelerine olanak tanıyan bir nesneye dönüştürmenin bir yolunu bulmamız gerekiyor. İçiniz rahatladı mı!🤩

Tidymodels'de, bu kullanışlı nesne [`workflow`](https://workflows.tidymodels.org/) olarak adlandırılır ve modelleme bileşenlerinizi pratik bir şekilde barındırır! Python'da buna *pipelines* derdik.

O halde her şeyi bir workflow içinde bir araya getirelim!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Üstelik, bir iş akışı tıpkı bir model gibi uygun hale getirilebilir/eğitilebilir.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Model çıktısından, eğitim sırasında öğrenilen katsayıları görebiliriz. Bu katsayılar, gerçek ve tahmin edilen değişken arasındaki toplam hatayı en aza indiren en iyi uyum çizgisinin katsayılarını temsil eder.

#### Test seti kullanarak model performansını değerlendirme

Modelin nasıl performans gösterdiğini görme zamanı 📏! Bunu nasıl yaparız?

Artık modeli eğittiğimize göre, test_set için tahminler yapmak için `parsnip::predict()` fonksiyonunu kullanabiliriz. Ardından, bu tahminleri gerçek etiket değerleriyle karşılaştırarak modelin ne kadar iyi (ya da kötü!) çalıştığını değerlendirebiliriz.

Hadi test seti için tahminler yaparak başlayalım ve ardından sütunları test setine bağlayalım.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Evet, bir model eğittiniz ve tahminler yapmak için kullandınız!🔮 Peki, bu model ne kadar iyi? Hadi modelin performansını değerlendirelim!

Tidymodels'de bunu `yardstick::metrics()` kullanarak yapıyoruz! Lineer regresyon için şu metriklere odaklanalım:

-   `Root Mean Square Error (RMSE)`: [MSE](https://en.wikipedia.org/wiki/Mean_squared_error)'nin karekökü. Bu, etiketle (bu durumda bir kabağın fiyatı) aynı birimde mutlak bir metrik sağlar. Değer ne kadar küçükse, model o kadar iyidir (basit bir anlamda, tahminlerin ortalama olarak ne kadar yanlış olduğunu temsil eder!)

-   `Coefficient of Determination (genellikle R-squared veya R2 olarak bilinir)`: Daha yüksek bir değerin daha iyi bir uyumu temsil ettiği göreceli bir metrik. Temelde, bu metrik modelin tahmin edilen ve gerçek etiket değerleri arasındaki varyansın ne kadarını açıklayabildiğini gösterir.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Model performansı düşüyor. Paket ve fiyatın bir dağılım grafiğini görselleştirerek daha iyi bir gösterge elde edip, ardından yapılan tahminleri kullanarak en iyi uyum çizgisini üzerine ekleyebilir miyiz, bir bakalım.

Bu, test setini hazırlayıp işleyerek paket sütununu kodlamamız ve ardından bunu modelimizin yaptığı tahminlerle birleştirmemiz gerektiği anlamına geliyor.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Harika! Gördüğünüz gibi, doğrusal regresyon modeli bir paketin fiyatı ile arasındaki ilişkiyi gerçekten iyi bir şekilde genelleştiremiyor.

🎃 Tebrikler, birkaç çeşit kabak fiyatını tahmin etmeye yardımcı olabilecek bir model oluşturdunuz. Tatil kabak bahçeniz harika görünecek. Ancak muhtemelen daha iyi bir model oluşturabilirsiniz!

## 5. Polinom regresyon modeli oluşturun

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalli tarafından hazırlanan bilgi grafiği</figcaption>


<!--![Dasani Madipalli tarafından hazırlanan bilgi grafiği](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Bazen verilerimiz doğrusal bir ilişkiye sahip olmayabilir, ancak yine de bir sonucu tahmin etmek isteyebiliriz. Polinom regresyon, daha karmaşık doğrusal olmayan ilişkiler için tahmin yapmamıza yardımcı olabilir.

Örneğin, kabak veri setimizdeki paket ve fiyat arasındaki ilişkiyi ele alalım. Bazen değişkenler arasında doğrusal bir ilişki olabilir - kabak hacmi büyüdükçe fiyatın artması gibi - ancak bazen bu ilişkiler bir düzlem veya doğru olarak çizilemez.

> ✅ İşte [polinom regresyon](https://online.stat.psu.edu/stat501/lesson/9/9.8) kullanabilecek verilere dair bazı örnekler
>
> Önceki grafikte Çeşit ve Fiyat arasındaki ilişkiye tekrar bir göz atın. Bu dağılım grafiği mutlaka bir doğru ile analiz edilmesi gereken bir ilişki gibi mi görünüyor? Belki de hayır. Bu durumda, polinom regresyonu deneyebilirsiniz.
>
> ✅ Polinomlar, bir veya daha fazla değişken ve katsayıdan oluşabilen matematiksel ifadelerdir.

#### Eğitim seti kullanarak bir polinom regresyon modeli eğitin

Polinom regresyon, doğrusal olmayan verilere daha iyi uyum sağlamak için *eğri bir çizgi* oluşturur.

Bir polinom modelinin tahmin yapmada daha iyi performans gösterip göstermeyeceğini görelim. Daha önce izlediğimiz prosedüre benzer bir yol izleyerek devam edeceğiz:

-   Verilerimizi modellemeye hazırlamak için uygulanması gereken ön işleme adımlarını belirten bir tarif oluşturun, örneğin: tahmin edicileri kodlama ve *n* dereceli polinomlar hesaplama

-   Bir model spesifikasyonu oluşturun

-   Tarif ve model spesifikasyonunu bir iş akışında birleştirin

-   İş akışını uydurarak bir model oluşturun

-   Modelin test verilerinde ne kadar iyi performans gösterdiğini değerlendirin

Haydi başlayalım!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Model performansını değerlendirin

👏👏Bir polinom modeli oluşturdunuz, şimdi test seti üzerinde tahminler yapalım!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, hadi `yardstick::metrics()` kullanarak modelin test_set üzerindeki performansını değerlendirelim.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Çok daha iyi performans.

`rmse` yaklaşık 7'den yaklaşık 3'e düştü, bu da gerçek fiyat ile tahmin edilen fiyat arasındaki hatanın azaldığını gösteriyor. Bunu *kabaca* şu şekilde yorumlayabilirsiniz: Ortalama olarak, yanlış tahminler yaklaşık \$3 kadar yanlıştır. `rsq` yaklaşık 0.4'ten 0.8'e yükseldi.

Tüm bu metrikler, polinom modelinin doğrusal modelden çok daha iyi performans gösterdiğini gösteriyor. Harika iş!

Hadi bunu görselleştirebilir miyiz bir bakalım!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Verilerinize daha iyi uyan bir eğri çizgisi görebilirsiniz! 🤩

Bunu daha da düzgün hale getirmek için `geom_smooth` fonksiyonuna bir polinom formülü geçirerek şu şekilde yapabilirsiniz:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Tıpkı pürüzsüz bir eğri gibi!🤩

İşte yeni bir tahmin yapmanın yolu:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


`polynomial model` tahmini, `price` ve `package` dağılım grafikleri göz önüne alındığında mantıklı görünüyor! Ve eğer bu model önceki modelden daha iyiyse, aynı verilere bakarak, bu daha pahalı kabaklar için bütçe ayırmanız gerekecek!

🏆 Tebrikler! Bir derste iki regresyon modeli oluşturdunuz. Regresyonun son bölümünde, kategorileri belirlemek için lojistik regresyonu öğreneceksiniz.

## **🚀Meydan Okuma**

Bu not defterinde birkaç farklı değişkeni test edin ve korelasyonun model doğruluğuyla nasıl ilişkili olduğunu görün.

## [**Ders sonrası test**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Gözden Geçirme ve Kendi Kendine Çalışma**

Bu derste Doğrusal Regresyon hakkında bilgi edindik. Regresyonun diğer önemli türleri de vardır. Stepwise, Ridge, Lasso ve Elasticnet teknikleri hakkında okuyun. Daha fazla bilgi edinmek için çalışabileceğiniz iyi bir kurs [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Harika Tidymodels çerçevesini nasıl kullanacağınızı öğrenmek istiyorsanız, lütfen aşağıdaki kaynaklara göz atın:

-   Tidymodels web sitesi: [Tidymodels ile Başlayın](https://www.tidymodels.org/start/)

-   Max Kuhn ve Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **TEŞEKKÜRLER:**

[R için daha sıcak ve ilgi çekici hale getiren harika illüstrasyonları oluşturan Allison Horst](https://twitter.com/allison_horst?lang=en). Daha fazla illüstrasyonu onun [galerisinde](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) bulabilirsiniz.



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
